#  Desafio Algar

Primeiro vamos instalar as bibliotecas para que possamos ustilizar o scikit-learn, SMOTE e XGBoostClassifier.

In [1]:
#!pip install scikit-learn --upgrade
#!pip install numpy  --upgrade
#!pip install pandas --upgrade
#!pip install imbalanced-learn --upgrade
!pip install imbalanced-learn==0.4.3 --upgrade
!pip install scikit-learn==0.20.0 --upgrade

     |████████████████████████████████| 174kB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 5.4MB 10.0MB/s eta 0:00:01
ERROR: autoai-libs 1.10.5 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement pandas>=0.24.2, but you'll have pandas 0.24.1 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement scikit-learn==0.20.3, but you'll have scikit-learn 0.20.1 which is incompatible.
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3


In [2]:
import pandas               as pd
from sklearn.model_selection  import train_test_split, GridSearchCV,\
                                     StratifiedKFold
from sklearn.preprocessing    import OneHotEncoder
from sklearn.metrics          import balanced_accuracy_score,\
                                     confusion_matrix, roc_auc_score
# Import xgboost
import xgboost              as xgb

from imblearn.over_sampling   import SMOTE
from imblearn.under_sampling  import RandomUnderSampler
from imblearn.pipeline        import Pipeline

Vamos ler o arquivo de dados fornecido com o desafio

In [3]:
# Lê o arquivo de dados
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c7e6595abc744e069b18b5eacdb9a257 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Z0WJnLqVWIhHc3pyswXI4LarKckCi0Ue2TmkUm9-kqrL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c7e6595abc744e069b18b5eacdb9a257.get_object(Bucket='mbtcdesafio4algar-donotdelete-pr-isjfn8wys52owr',Key='algar-dataset-treino.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)

print(df.info(), '\n')
print('Tipo de Campos e quantidade de colunas\n', df.dtypes.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370 entries, 0 to 1369
Data columns (total 35 columns):
Idade                                         1370 non-null int64
Local de trabalho                             1370 non-null object
Pontuação teste                               1370 non-null int64
Departmento                                   1370 non-null object
Distancia casa-trabalho                       1370 non-null int64
Educacao                                      1370 non-null object
Area                                          1370 non-null object
Possui carro                                  1370 non-null int64
Subordinado                                   1370 non-null int64
Satisfação com o ambiente no emprego atual    1370 non-null int64
Genero                                        1370 non-null object
Horas voluntariado                            1370 non-null int64
Envolvimento com trabalho                     1370 non-null int64
Posicao                       

Notamos que o arquivo é composto por colunas dos tipos inteiro e alfanumérico.

In [4]:
# Columns ith one value
a= [col for col in df.columns if df[col].value_counts().count()<2]
print('Columns with unique value:', a)

Columns with unique value: ['Possui carro', 'Maior de idade', 'Horas de trabalho padrão']


In [5]:
# Subordinado has unique values
print(df['Subordinado'].value_counts().count() == len(df['Subordinado']))

True


In [6]:
# Get Features and target
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

# Eliminate columns with unique feature values
X = X.drop(a + ['Subordinado'], axis=1)

# Categorical Columns
c_columns = X.select_dtypes(include='object')
nc_columns = [col for col in X.columns if col not in c_columns.columns]

ohe = OneHotEncoder(handle_unknown='ignore')

c_columns = pd.DataFrame(ohe.fit_transform(c_columns).toarray())
c_columns.columns = ohe.get_feature_names()

X = pd.concat([X[nc_columns], c_columns], axis=1)

In [7]:
# Train test split  75% / 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=555)

In [8]:
# Assemble Model

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', random_state=123)

# tunning parameters
gbm_param_grid = [
    {
     'SMOTE__sampling_strategy': [0.5, 0.4, 0.3],
     'under__sampling_strategy': [0.5, 0.6, 0.7],
     'model__colsample_bytree': [0.055],
     'model__n_estimators': [130, 150, 200],
     'model__max_depth': [1, 2, 3],
     'model__reg_alpha': [0, 0.25, 0.3, 0.35],
     'model__learning_rate': [0.3, 0.35, 0.5, 0.6]
    }
]
# Instantiate the classifier: gbm
gbm = xgb.XGBClassifier()
over = SMOTE(sampling_strategy=0.5)
under = RandomUnderSampler(sampling_strategy=0.5, random_state=555)
kfold = StratifiedKFold(n_splits=5, shuffle=False)

# Assamble Pipeline
steps = [('SMOTE', over),
         ('under', under), 
         ('model', gbm)]

pipeline = Pipeline(steps=steps)

# Perform grid search: grid_mse
grid_acc = GridSearchCV(param_grid=gbm_param_grid, estimator=pipeline,
                        scoring = "balanced_accuracy", cv=kfold, verbose=1,
                        n_jobs=2)

In [9]:
#Tune Model
# Fit grid_mse to the data
grid_acc.fit(X_train, y_train)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54'] ['Idade', 'Pontuação teste', 'Distancia casa-trabalho', 'Satisfação com o ambiente no emprego atual', 'Horas voluntariado', 'Envolvimento com trabalho', 'Posicao', 'Satisfação com emprego', 'Renda', 'Bonus de performance', 'Quantidade de empresas que trabalho', 'Aumento de salario%', 'Performance na entrevista', 'Satisfação com a relação', 'Beneficios', 'Anos de experiencia', 'Horas de treinamento ultimo ano', 'Estilo de vida', 'Anos na última empresa', 'Anos na posição atual', 'Anos desde última promoção', 'Anos com a mesma gerência', 'x0_Cliente', 'x0_Escritório', 'x0_Misto', 'x1

In [ ]:
# Print scores
# Print the best parameters and lowest RMSE
print("\nBest parameters found: ", grid_acc.best_params_)
print("\nHighest average balanced accuracy found: %.4f" % grid_acc.best_score_)

print('\nBest Estimator: \n', grid_acc.best_estimator_)

print(' \nBest test score: %f' % (grid_acc.score(X_test, y_test)))

#Print scores
print('\nTrain score: %f' % grid_acc.score(X_train, y_train))
print('Test  score: %f\n' % grid_acc.score(X_test, y_test))

y_pred = grid_acc.predict(X_test)
ba = balanced_accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred, labels=['Não', 'Sim'])
ac = cm[0,0]/(cm[0,0]+cm[0,1])
sp = cm[1,1]/(cm[1,1]+cm[1,0])
auc = roc_auc_score(y_test=='Não', y_pred=='Não')

print('\nBalanced Accuracy : %.4f' % ba)
print('AUC               : %.4f' % auc)
print('Accuracy          : %.4f' % ac)
print('Specificity       : %.4f\n' % sp)

### Encapsulando uma Pipeline personalizada no Watson Machine Learning

In [10]:
#!pip install watson_machine_learning_client --upgrade
#!pip install joblib --upgrade
!pip install imbalanced-learn==0.4.3 --upgrade
!pip install scikit-learn==0.20.1 --upgrade

Requirement already up-to-date: imbalanced-learn==0.4.3 in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.4.3)
Requirement already up-to-date: scikit-learn==0.20.1 in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.20.1)


In [11]:
import xgboost              as xgb

from imblearn.over_sampling   import SMOTE
from imblearn.under_sampling  import RandomUnderSampler
from imblearn.pipeline        import Pipeline

In [12]:
# Assamble Best Model Pipeline
pipeline = Pipeline(steps=[('SMOTE', SMOTE(sampling_strategy=0.3)),
                           ('under', RandomUnderSampler(random_state=555, sampling_strategy=0.7)),
                           ('model', xgb.XGBClassifier(objective='binary:logistic',
                                                       colsample_bytree=0.055, learning_rate=0.6,
                                                       max_depth=1, n_estimators=150, reg_alpha=0.25))
                          ])

In [13]:
# Fit Pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('SMOTE', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=None, ratio=None,
   sampling_strategy=0.3, svm_estimator='deprecated')), ('under', RandomUnderSampler(random_state=555, ratio=None, replacement=False,
          return...g_alpha=0.25,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))])

#### Estabelecendo conexão entre o cliente Python do WML e a sua instância do serviço na nuvem

In [14]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-08-27 20:43:28,284 - watson_machine_learning_client.wml_client_error - WARNING - Deployment creation failed. Error: 400. {"trace":"-1iz0y28oegmas","errors":[{"code":"unsupported_framework_details","message":"The xgboost version 0.80 is not supported with scikit-learn version 0.20 . Expected scikit-learn version here is 0.19 . Retry deployment after saving model with expected scikit-learn version."}]}


As próximas células irão realizar o deploy da pipeline declarada neste notebook no WML. Só prossiga se você já está satisfeito com seu modelo e acha que já é a hora de fazer o deploy da sua solução.

Cole as credenciais de sua instância do Watson Machine Learning na variável na célula abaixo.

É importante que a variável que contém os valores tenha o nome de ``wml_credentials`` para que as próximas células deste notebook executem corretamente.

In [15]:
wml_credentials = {
    "apikey": "UcKj40CKGjKuynWWqaQQM9R8k7mNw-1UW6FAEZ5RtGSw",
    "iam_apikey_description": "Auto-generated for key d7837118-bdd4-4b23-9b87-f6752bb14446",
    "iam_apikey_name": "wdp-writer",
    "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
    "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/47c20058f2c9403d8fdd36abe258ae75::serviceid:ServiceId-68885698-5eef-403f-a93d-51812538b1f2",
    "instance_id": "c617f0af-1f55-43c1-a23d-608de97d9765",
    "url": "https://us-south.ml.cloud.ibm.com"
}

In [16]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [17]:
# Extraindo detalhes da sua instância do Watson Machine Learning
import json
instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

{
    "entity": {
        "source": "Bluemix",
        "published_models": {
            "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c617f0af-1f55-43c1-a23d-608de97d9765/published_models"
        },
        "usage": {
            "capacity_units": {
                "current": 0,
                "limit": 180000000
            },
            "computation_time": {
                "current": 0,
                "limit": 180000
            },
            "deployment_count": {
                "current": 0,
                "limit": 5
            },
            "expiration_date": "2020-09-01T00:00:00.000Z",
            "gpu_count_k80": {
                "current": 0,
                "limit": 8
            },
            "gpu_count_p100": {
                "current": 0,
                "limit": 0
            },
            "gpu_count_v100": {
                "current": 0,
                "limit": 0
            },
            "model_count": {
                "current": 0,
         

**ATENÇÃO!!**

Fique atento para os limites de consumo de sua instância do Watson Machine Learning!

Caso você expire a camada grátis, não será possível avaliar seu modelo (pois é necessária a realização de algumas chamadas de API que consomem predições!)

#### Listando todos os artefatos armazenados no seu WML

Para listar todos os artefatos armazenados em seu Watson Machine Learning, você pode usar a seguinte função:

    clientWML.repository.list()

In [18]:
# Listando todos os artefatos atualmente armazenados na sua instância do WML

clientWML.repository.list()

------------------------------------  -----------------------  ------------------------  ---------  --------------
GUID                                  NAME                     CREATED                   FRAMEWORK  TYPE
b1ad06e6-41f6-416e-a158-5b2834bc5cb2  my_custom_wml_runtime_1  2020-08-27T20:23:56.476Z  -          python runtime
------------------------------------  -----------------------  ------------------------  ---------  --------------


No plano LITE do Watson Machine Learning só é permitido o deploy de um único modelo por vez. Se for o caso de você já possuir um modelo online na sua instância, você pode apagá-lo utilizando o método clientWML.repository.delete():

    artifact_guid = "359c8951-d2fe-4063-8706-cc06b32d5e0d"
    clientWML.repository.delete(artifact_guid)

#### Criando uma nova definição de pacote Python personalizado no WML

O primeiro passo para realizar seu deploy é armazenar o código das transformações personalizadas criadas por você.

Para essa etapa precisamos apenas do arquivo .zip do pacote criado (que já possuimos carregado no Kernel!)

In [19]:
# Definição de metadados do nosso pacote com as Transforms personalizadas
#pkg_meta = {
#    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_1",
#    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
#    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip criado anteriormente!
#    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
#    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
#}
#custom_package_details = clientWML.runtimes.store_library( pkg_meta )
#custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

artifact_guid = "b1ad06e6-41f6-416e-a158-5b2834bc5cb2"
clientWML.repository.delete(artifact_guid)

print("\n Lista de artefatos de runtime armazenados no WML:")
clientWML.repository.list()


 Lista de artefatos de runtime armazenados no WML:
----  ----  -------  ---------  ----
GUID  NAME  CREATED  FRAMEWORK  TYPE
----  ----  -------  ---------  ----


#### Criando uma nova definição de runtime Python personalizado no WML

O segundo passo é armazenar uma definição de runtime Python para utilizar a nossa biblioteca personalizada.

Isso pode ser feito da seguinte forma:

In [20]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_1",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python 3.6 runtime",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    #clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalhes do runtime armazenado:")
print(json.dumps(runtime_details, indent=4))


 Detalhes do runtime armazenado:
{
    "entity": {
        "name": "my_custom_wml_runtime_1",
        "description": "A Python 3.6 runtime",
        "custom_libraries": [],
        "content_url": "https://private.us-south.ml.cloud.ibm.com/v4/runtimes/935b20e4-cd2c-45f7-a716-78e6d719312c/content",
        "platform": {
            "name": "python",
            "version": "3.6"
        }
    },
    "metadata": {
        "created_at": "2020-08-27T20:43:00.377Z",
        "guid": "935b20e4-cd2c-45f7-a716-78e6d719312c",
        "url": "https://us-south.ml.cloud.ibm.com/v4/runtimes/935b20e4-cd2c-45f7-a716-78e6d719312c"
    }
}


In [21]:
# Listando todos runtimes armazenados no seu WML:
clientWML.runtimes.list()

------------------------------------  -----------------------  ------------------------  ----------
GUID                                  NAME                     CREATED                   PLATFORM
935b20e4-cd2c-45f7-a716-78e6d719312c  my_custom_wml_runtime_1  2020-08-27T20:43:00.377Z  python-3.6
------------------------------------  -----------------------  ------------------------  ----------


#### Criando uma nova definição de Pipeline personalizada no WML

Finalmente iremos criar uma definição (metadados) para a nossa Pipeline ser hospedada no WML.

Definimos como parâmetros um nome para o artefato e o ID do runtime criado anteriormente.

In [22]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-4-mbtc2020-pipeline-1',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Em seguida chamamos o método para armazenar a nova definição:

In [23]:
# Função para armazenar uma definição de Pipeline no WML
stored_model_details = clientWML.repository.store_model(
    model=pipeline,  # `my_pipeline` é a variável criada anteriormente e contém nossa Pipeline já treinada :)
    meta_props=model_meta,  # Metadados definidos na célula anterior
    training_data=None  # Não altere esse parâmetro
)

print("\n Lista de artefatos armazenados no WML:")
clientWML.repository.list()

# Detalhes do modelo hospedado no Watson Machine Learning
print("\n Metadados do modelo armazenado:")
print(json.dumps(stored_model_details, indent=4))


 Lista de artefatos armazenados no WML:
------------------------------------  -----------------------------  ------------------------  -----------------  --------------
GUID                                  NAME                           CREATED                   FRAMEWORK          TYPE
c3d7f95d-c934-4a80-93c9-105f3d4c64e9  desafio-4-mbtc2020-pipeline-1  2020-08-27T20:43:16.855Z  scikit-learn-0.20  model
935b20e4-cd2c-45f7-a716-78e6d719312c  my_custom_wml_runtime_1        2020-08-27T20:43:00.377Z  -                  python runtime
------------------------------------  -----------------------------  ------------------------  -----------------  --------------

 Metadados do modelo armazenado:
{
    "metadata": {
        "guid": "c3d7f95d-c934-4a80-93c9-105f3d4c64e9",
        "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c617f0af-1f55-43c1-a23d-608de97d9765/published_models/c3d7f95d-c934-4a80-93c9-105f3d4c64e9",
        "created_at": "2020-08-27T20:43:16.855Z",
        "modi

#### Realizando o deployment do seu modelo para consumo imediato por outras aplicações

In [24]:
# O deployment do modelo é finalmente realizado por meio do método ``deployments.create()``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # Não altere esse parâmetro
    name="desafio-4-mbtc2020-deployment-1",
    description="Solução do desafio 4 - MBTC",
    asynchronous=False,  # Não altere esse parâmetro
    deployment_type='online',  # Não altere esse parâmetro
    deployment_format='Core ML',  # Não altere esse parâmetro
    meta_props=model_meta  # Não altere esse parâmetro
)

{"trace":"-1iz0y28oegmas","errors":[{"code":"unsupported_framework_details","message":"The xgboost version 0.80 is not supported with scikit-learn version 0.20 . Expected scikit-learn version here is 0.19 . Retry deployment after saving model with expected scikit-learn version."}]}


--------------------------
Deployment creation failed
--------------------------




WMLClientError: Deployment creation failed. Error: 400. {"trace":"-1iz0y28oegmas","errors":[{"code":"unsupported_framework_details","message":"The xgboost version 0.80 is not supported with scikit-learn version 0.20 . Expected scikit-learn version here is 0.19 . Retry deployment after saving model with expected scikit-learn version."}]}

#### Testando um modelo hospedado no Watson Machine Learning

In [ ]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("A URL de chamada da sua API é: {}".format(model_endpoint_url))

In [ ]:
# Detalhes do deployment realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # esse é o ID do seu deployment!
)

print("Metadados do deployment realizado: \n")
print(json.dumps(deployment_details, indent=4))

In [ ]:
scoring_payload = {
    'fields': [
        "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
        "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    ],
    'values': [
        [
            513949,"Marli Quésia de Oliveira",1,1,1,1,4.3,4.0,3.1,4.9,0,3,4,3,
        ]
    ]
}

print("\n Payload de dados a ser classificada:")
print(json.dumps(scoring_payload, indent=4))

In [ ]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))

<hr>

## Parabéns! 

Se tudo foi executado sem erros, você já tem um classificador baseado em machine learning encapsulado como uma API REST!

Para testar a sua solução integrada com um assistente virtual e realizar a submissão, acesse a página:

https://uninassau.maratona.dev

Você irá precisar da endpoint url do seu modelo e das credenciais do WML :)